## __workshare__ Construct

The following are examples of the __workshare__ construct.

In the following example, __workshare__ spreads work across the threads executing  the __parallel__ region, and there is a barrier after the last statement.  Implementations must enforce Fortran execution rules inside of the __workshare__  block.

In [ ]:
!!%compiler: gfortran
!!%cflags: -fopenmp

! name: workshare.1
! type: F-fixed
      SUBROUTINE WSHARE1(AA, BB, CC, DD, EE, FF, N)
      INTEGER N
      REAL AA(N,N), BB(N,N), CC(N,N), DD(N,N), EE(N,N), FF(N,N)

!$OMP    PARALLEL
!$OMP     WORKSHARE
            AA = BB
            CC = DD
            EE = FF
!$OMP     END WORKSHARE
!$OMP   END PARALLEL

      END SUBROUTINE WSHARE1

In the following example, the barrier at the end of the first __workshare__  region is eliminated with a __nowait__ clause. Threads doing __CC =  DD__ immediately begin work on __EE = FF__ when they are done with __CC  = DD__.

In [ ]:
!!%compiler: gfortran
!!%cflags: -fopenmp

! name: workshare.2
! type: F-fixed
      SUBROUTINE WSHARE2(AA, BB, CC, DD, EE, FF, N)
      INTEGER N
      REAL AA(N,N), BB(N,N), CC(N,N)
      REAL DD(N,N), EE(N,N), FF(N,N)

!$OMP   PARALLEL
!$OMP     WORKSHARE
            AA = BB
            CC = DD
!$OMP     END WORKSHARE NOWAIT
!$OMP     WORKSHARE
            EE = FF
!$OMP     END WORKSHARE
!$OMP   END PARALLEL
       END SUBROUTINE WSHARE2

The following example shows the use of an __atomic__ directive inside a __workshare__  construct. The computation of __SUM(AA)__ is workshared, but the update to  __R__ is atomic.

In [ ]:
!!%compiler: gfortran
!!%cflags: -fopenmp

! name: workshare.3
! type: F-fixed
      SUBROUTINE WSHARE3(AA, BB, CC, DD, N)
      INTEGER N
      REAL AA(N,N), BB(N,N), CC(N,N), DD(N,N)
      REAL R
        R=0
!$OMP   PARALLEL
!$OMP     WORKSHARE
            AA = BB
!$OMP       ATOMIC UPDATE
              R = R + SUM(AA)
            CC = DD
!$OMP     END WORKSHARE
!$OMP   END PARALLEL
      END SUBROUTINE WSHARE3

Fortran __WHERE__ and __FORALL__ statements are **compound statements**,  made up of a **control** part and a **statement** part. When __workshare__  is applied to one of these compound statements, both the control and the statement  parts are workshared. The following example shows the use of a __WHERE__ statement  in a __workshare__ construct.

Each task gets worked on in order by the threads:

__AA = BB__ then <br> __CC = DD__ then <br> __EE .ne. 0__ then <br> __FF = 1 / EE__ then <br> __GG = HH__

In [ ]:
!!%compiler: gfortran
!!%cflags: -fopenmp

! name: workshare.4
! type: F-fixed
      SUBROUTINE WSHARE4(AA, BB, CC, DD, EE, FF, GG, HH, N)
      INTEGER N
      REAL AA(N,N), BB(N,N), CC(N,N)
      REAL DD(N,N), EE(N,N), FF(N,N)
      REAL GG(N,N), HH(N,N)

!$OMP   PARALLEL
!$OMP     WORKSHARE
            AA = BB
            CC = DD
            WHERE (EE .ne. 0) FF = 1 / EE
            GG = HH
!$OMP     END WORKSHARE
!$OMP   END PARALLEL

      END SUBROUTINE WSHARE4

In the following example, an assignment to a shared scalar variable is performed  by one thread in a __workshare__ while all other threads in the team wait.

In [ ]:
!!%compiler: gfortran
!!%cflags: -fopenmp

! name: workshare.5
! type: F-fixed
      SUBROUTINE WSHARE5(AA, BB, CC, DD, N)
      INTEGER N
      REAL AA(N,N), BB(N,N), CC(N,N), DD(N,N)

        INTEGER SHR

!$OMP   PARALLEL SHARED(SHR)
!$OMP     WORKSHARE
            AA = BB
            SHR = 1
            CC = DD * SHR
!$OMP     END WORKSHARE
!$OMP   END PARALLEL

      END SUBROUTINE WSHARE5

The following example contains an assignment to a private scalar variable, which  is performed by one thread in a __workshare__ while all other threads wait.  It is non-conforming because the private scalar variable is undefined after the  assignment statement.

In [ ]:
!!%compiler: gfortran
!!%cflags: -fopenmp

! name: workshare.6
! type: F-fixed
      SUBROUTINE WSHARE6_WRONG(AA, BB, CC, DD, N)
      INTEGER N
      REAL AA(N,N), BB(N,N), CC(N,N), DD(N,N)

        INTEGER PRI

!$OMP   PARALLEL PRIVATE(PRI)
!$OMP     WORKSHARE
            AA = BB
            PRI = 1
            CC = DD * PRI
!$OMP     END WORKSHARE
!$OMP   END PARALLEL

      END SUBROUTINE WSHARE6_WRONG

Fortran execution rules must be enforced inside a __workshare__ construct.  In the following example, the same result is produced in the following program  fragment regardless of whether the code is executed sequentially or inside an OpenMP  program with multiple threads:

In [ ]:
!!%compiler: gfortran
!!%cflags: -fopenmp

! name: workshare.7
! type: F-fixed
      SUBROUTINE WSHARE7(AA, BB, CC, N)
      INTEGER N
      REAL AA(N), BB(N), CC(N)

!$OMP   PARALLEL
!$OMP     WORKSHARE
            AA(1:50)  = BB(11:60)
            CC(11:20) = AA(1:10)
!$OMP     END WORKSHARE
!$OMP   END PARALLEL

      END SUBROUTINE WSHARE7